In [1]:
!pip install -qq implicit

     |████████████████████████████████| 18.5 MB 552 kB/s 


In [43]:
import implicit
implicit.__version__

'0.5.2'

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/My Drive/webinar_rec2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
def items_weights_creator(df):
    
    sales_count = df.groupby('item_id')['quantity'].count().reset_index().sort_values('quantity', ascending=False)
    
    sales_count.rename(columns = {'quantity':'n_sold'}, inplace=True)
    
    sales_count['weight'] = np.log(sales_count['n_sold'])/sum(np.log(sales_count['n_sold']))
    
    sales_count.drop('n_sold', axis=1, inplace=True)
        
    return sales_count

In [10]:
def get_item_weights(df, pop_type='quantity', eps=1e-10):
    if pop_type == 'quantity_fact':
        popularity = df.groupby('item_id')['quantity'].count().reset_index()
        popularity.rename(columns={'quantity': 'popularity'}, inplace=True)
        popularity['weight'] = popularity['popularity'] / len(df)
    elif pop_type == 'quantity_log':
        popularity = df.groupby('item_id')['quantity'].count().reset_index()
        popularity.rename(columns={'quantity': 'popularity'}, inplace=True)
        popularity['weight'] = np.log(popularity['popularity']) / np.sum(np.log(popularity['popularity']))
    elif pop_type == 'quantity':
        popularity = df.groupby('item_id')['quantity'].sum().reset_index()
        popularity.rename(columns={'quantity': 'popularity'}, inplace=True)
        popularity['weight'] = popularity['popularity'] / np.sum(popularity['popularity'])
    elif pop_type == 'sales':
        popularity = df.groupby('item_id')['sales_value'].sum().reset_index()
        popularity.rename(columns={'sales_value': 'popularity'}, inplace=True)
        weights = np.log(popularity.popularity*100 + eps)
        weights[weights<0] = 0
        popularity['weight'] = weights / np.sum(weights)
    else:
        raise ValueError(f'Wrong type of popularity type')
    popularity.drop('popularity', axis=1, inplace=True)
    
    return popularity

In [64]:
quantity_popularity = get_item_weights(data_train, pop_type='quantity_log')

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [13]:
result = pd.read_csv('/content/drive/My Drive/webinar_rec2/predictions_basic2.csv')

In [65]:
%%time

result['weighted_random_recommendation'] = result['user_id'].\
                          apply(lambda x: weighted_random_recommendation(quantity_popularity, n=5))

CPU times: user 310 ms, sys: 3.97 ms, total: 314 ms
Wall time: 316 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [15]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [16]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[1409814, 1111343, 8157378, 873911, 1081751]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 1127831, 1082185, 999999, 981760]","[1098066, 1127831, 981760, 999999, 1082185]","[1098066, 999999, 1127831, 981760, 1082185]","[1081177, 995785, 1004906, 1082185, 1029743, 9...","[847573, 1061732, 1087366, 1120443, 201376]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[13002761, 329987, 6395533, 13506316, 1134871]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 1127831, 1082185, 999999, 981760]","[1098066, 1127831, 981760, 999999, 1082185]","[1098066, 999999, 1127831, 981760, 1082185]","[1081177, 995785, 1004906, 1082185, 1029743, 9...","[1133827, 1045220, 12582269, 5565905, 896666]"


##### Reconstruct data dtypes in result

In [18]:
result.actual = result.actual.apply(lambda x: np.array(x.split()[1:-1]).astype(int))

In [19]:
for col in result.columns[2:-1]:
    result[col] = result[col].apply(lambda x: eval(x))

##### Calculate mean precision@k, k = 5

In [20]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.000881488736532811

In [21]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.14848188050930464

In [22]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.13300685602350637

In [23]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

0.11351616062683643

In [24]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.11351616062683643

In [25]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

0.1603330068560235

In [26]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.0019588638589618022

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [27]:
top_5000 = data_train.groupby('item_id')['quantity'].sum().reset_index().sort_values('quantity', ascending=False).head(5000)

In [28]:
data_5000 = data_train.loc[top_5000.index, :]

In [29]:
quantity_popularity_5000 = get_item_weights(data_5000)

In [30]:
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(quantity_popularity_5000, n=5))

In [31]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation_5000'], row['actual']), axis=1).mean()

0.0494613124387855

In [32]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [33]:
items = data_5000.item_id.unique()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [34]:
result.apply(lambda row: precision_at_k(row['random_recommendation_5000'], row['actual']), axis=1).mean()

0.004505386875612145

In [35]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [36]:
popular_recs = popularity_recommendation(data_5000, n=5)

result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popular_recs)

In [37]:
result.apply(lambda row: precision_at_k(row['popular_recommendation_5000'], row['actual']), axis=1).mean()

0.1670910871694417

##### ItemItem Deterministic models

In [38]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [39]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [40]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
sparse_item_user = sparse_user_item.T.tocsr()

In [41]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

##### For-loop for K-neighbors selection

In [52]:
best_score = -np.inf
best_K = 0

for j in range(2, 16):
    model = ItemItemRecommender(K=j, num_threads=4)
    model.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=False)
    result[f'itemitem_K{j}'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[i] for i in model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]])
    score = result.apply(lambda row: precision_at_k(row[f'itemitem_K{j}'], row['actual']), axis=1).mean()
    if score > best_score:
        best_score = score
        best_K = j
print(f'Best precision@5: {best_score*100:2.4f}, number of neighbors: {best_K}.')

Best precision@5: 19.0891, number of neighbors: 3.


In [55]:
best_score = -np.inf
best_K = 0

for j in range(1, 16):
    model = CosineRecommender(K=j, num_threads=4)
    model.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=False)
    result[f'cosine_K{j}'] = result['user_id'].\
                              apply(lambda x: [id_to_itemid[i] for i in model.recommend(userid=userid_to_id[x], 
                                                                                        user_items=sparse_user_item,
                                                                                        N=5, 
                                                                                        filter_already_liked_items=False, 
                                                                                        filter_items=[itemid_to_id[999999]], 
                                                                                        recalculate_user=True)[0]])
    score = result.apply(lambda row: precision_at_k(row[f'cosine_K{j}'], row['actual']), axis=1).mean()
    if score > best_score:
        best_score = score
        best_K = j
print(f'Best precision@5: {best_score*100:2.4f}, number of neighbors: {best_K}.')

Best precision@5: 14.4858, number of neighbors: 12.


In [56]:
best_score = -np.inf
best_K = 0

for k in range(1, 16):
    model = TFIDFRecommender(K=k, num_threads=4)
    model.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=False)
    result[f'tfidf_K{k}'] = result['user_id'].\
                            apply(lambda x: [id_to_itemid[i] for i in model.recommend(userid=userid_to_id[x], 
                                                                                      user_items=sparse_user_item,
                                                                                      N=5, 
                                                                                      filter_already_liked_items=False, 
                                                                                      filter_items=[itemid_to_id[999999]], 
                                                                                      recalculate_user=True)[0]])
    score = result.apply(lambda row: precision_at_k(row[f'tfidf_K{k}'], row['actual']), axis=1).mean()
    if score > best_score:
        best_score = score
        best_K = k
print(f'Best precision@5: {best_score*100:2.4f}, number of neighbors: {best_K}.')

Best precision@5: 14.4858, number of neighbors: 3.
